<a href="https://colab.research.google.com/github/ImperatorAvrelianvs/Generalized_Pell_equation_solver/blob/main/Pell_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Colab Monte‑Carlo Simulation of Generalized Pell (including zero‐solution cases)
import math
import random
import signal
import pandas as pd
from typing import List, Tuple

# --- 0) Timeout setup ---
class TimeoutError(Exception):
    """Raised when a per-(D,N) solve exceeds the time limit."""
    pass

def _timeout_handler(signum, frame):
    raise TimeoutError()

signal.signal(signal.SIGALRM, _timeout_handler)

# --- 1) Continued‑fraction PQa generator ---
def continued_fraction_pqa(D: int, P0: int, Q0: int):
    P, Q = P0, Q0
    A2, A1 = 0, 1
    B2, B1 = 1, 0
    G2, G1 = -P0, Q0
    i = 0
    while True:
        a = (P + math.isqrt(D)) // Q
        A = a*A1 + A2
        B = a*B1 + B2
        G = a*G1 + G2
        if i >= 1 and abs(Q) == 1:
            yield G1, B1
            return
        Pn = a*Q - P
        Qn = (D - Pn*Pn) // Q
        A2, A1 = A1, A
        B2, B1 = B1, B
        G2, G1 = G1, G
        P, Q = Pn, Qn
        i += 1

# --- 2) Solve t^2 − D u^2 = −1 if needed ---
def solve_pell_minus1(D: int) -> Tuple[int,int]:
    a0 = math.isqrt(D)
    m, d, a = 0, 1, a0
    h2, h1 = 1, a
    k2, k1 = 0, 1
    while True:
        if h1*h1 - D*k1*k1 == -1:
            return h1, k1
        m, d = d*a - m, (D - (d*a - m)**2)//d
        a = (a0 + m)//d
        h2, h1 = h1, a*h1 + h2
        k2, k1 = k1, a*k1 + k2

# --- 3) LMM solver for x^2 − D y^2 = N (primitive per class) ---
def generalized_pell_LMM(D: int, N: int) -> List[Tuple[int,int]]:
    sols = []
    absN = abs(N)
    if absN == 0:
        return [(0,0)]
    for f in range(1, math.isqrt(absN) + 1):
        if N % (f*f) != 0:
            continue
        m = N // (f*f)
        mod = abs(m)
        for z in range(-mod//2 + 1, mod//2 + 1):
            if (z*z - D) % mod != 0:
                continue
            for r, s in continued_fraction_pqa(D, z, mod):
                val = r*r - D*s*s
                if   val ==  m:
                    sols.append((f*r, f*s))
                elif val == -m:
                    mn = solve_pell_minus1(D)
                    if mn:
                        t, u = mn
                        x = f*(r*t + s*u*D)
                        y = f*(r*u + s*t)
                        sols.append((x, y))
                break
    return list(set(sols))

# --- 4) Fundamental unit for x^2 − D y^2 = 1 ---
def solve_pell_unit(D: int) -> Tuple[int,int]:
    a0 = math.isqrt(D)
    m, d, a = 0, 1, a0
    h2, h1 = 1, a
    k2, k1 = 0, 1
    while True:
        if h1*h1 - D*k1*k1 == 1:
            return h1, k1
        m, d = d*a - m, (D - (d*a - m)**2)//d
        a = (a0 + m)//d
        h2, h1 = h1, a*h1 + h2
        k2, k1 = k1, a*k1 + k2

# --- 5) Simulation parameters ---
TRIALS        = 10000     # number of (D,N) samples
class_length  = 10      # # extended solutions per class
TIMEOUT       = 2       # seconds per (D,N) solve

# --- 6) Monte Carlo loop ---
rows = []
for _ in range(TRIALS):
    D = random.choice([d for d in range(2, 201) if math.isqrt(d)**2 != d])
    N = random.randint(-200, 200)

    signal.alarm(TIMEOUT)
    try:
        primitives = generalized_pell_LMM(D, N)
    except TimeoutError:
        primitives = []
    finally:
        signal.alarm(0)

    num_classes = len(primitives)

    if num_classes == 0:
        # record a zero‐solution case
        rows.append({
            'D': D,
            'N': N,
            'num_classes': 0,
            'x': 0,
            'y': 0
        })
        continue

    # fundamental unit for extension
    u, v = solve_pell_unit(D)

    # for each primitive, generate up to class_length extended solutions
    for x0, y0 in primitives:
        uk, vk = 1, 0
        for _ in range(class_length):
            xk = x0*uk + y0*vk*D
            yk = x0*vk + y0*uk
            rows.append({
                'D': D,
                'N': N,
                'num_classes': num_classes,
                'x': xk,
                'y': yk
            })
            uk, vk = uk*u + vk*v*D, uk*v + vk*u

# --- 7) Save to CSV ---
df = pd.DataFrame(rows)
csv_path = '/content/pell_dataset.csv'
df.to_csv(csv_path, index=False)

print("Done! Sample of generated data:")
print(df.head())
print("\nDataset saved to:", csv_path)


Done! Sample of generated data:
     D    N  num_classes  x  y
0  113  -87            0  0  0
1  140  136            0  0  0
2  188  -20            0  0  0
3   60 -118            0  0  0
4   31 -122            0  0  0

Dataset saved to: /content/pell_dataset.csv


In [8]:
from google.colab import files
files.download('/content/pell_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>